## Testing installed packages via Conda 

We will first ensure that the installed packages are working properly. You should be able to run the following cells. Once everything passes, continue onto the geometry assignment. 

In [2]:
try:
    from jax import config
    config.update("jax_enable_x64", True)
    import meshcat
    import meshcat.geometry as geom
    import math
    import meshcat.transformations as tfm
    import numpy as onp
    import time
    import jax.numpy as np
    import jax
    from jax import jacfwd, hessian
    from jaxlie import SE2, SE3, SO3
    import matplotlib.pyplot as plt
    from viewer import DoublePendViewer, BlockRigidBodyViewer

    print('Import packages works! Great work following directions :D !')
except Exception as e:
    print('Something went wrong. The following error tells you what happened. Go through README.md again and see what went wrong')
    print(e)

Something went wrong. The following error tells you what happened. Go through README.md again and see what went wrong
No module named 'viewer'


## Meshcat Visualization Tool
Meshcat is a WebGL based 3D renderer that works on your web browser and is based in javascript. We will be using this visualization tool to plot reference frames, perform transforms, and build robot visualizations. Run the code below to familiarize yourself with `meshcat-python` which is a wrapper around meshcat in python. You can check out more examples at the [github page](https://github.com/meshcat-dev/meshcat-python/tree/master/examples) . 


In [3]:
# Create a new visualizer
vis = meshcat.Visualizer()

# Create a jupyter cell that renders the visalizer by calling the function below
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [4]:
# Finger properties

MASS_g_PROXIMAL = 13.235
MASS_g_DISTAL = 10.628

MASS_kg_PROXIMAL = 0.013235
MASS_kg_DISTAL = 0.010628

# PROXIMAL_SPRING_STIFFNESS = 0.49
# DISTAL_SPRING_STIFFNESS = 0.178
PROXIMAL_SPRING_STIFFNESS = 0.178
DISTAL_SPRING_STIFFNESS = PROXIMAL_SPRING_STIFFNESS * 3

In [5]:
# Finger geometry
vis.delete()

PROXIMAL_LENGTH = 0.625
DISTAL_LENGTH = 0.447
FINGER_RADIUS = 0.08

BASE_2_PROXIMAL = tfm.translation_matrix([0, 0, 0])
proximal_rot = tfm.euler_matrix(0, 0, 0)
PROXIMAL_2_DISTAL_1 = tfm.translation_matrix([0.0, 0, PROXIMAL_LENGTH/2]) @ tfm.euler_matrix(np.pi/2, 0, 0)
PROXIMAL_2_DISTAL_2 = tfm.translation_matrix([0.0, PROXIMAL_LENGTH/2, 0])
distal_rot = tfm.euler_matrix(0, 0, 0)
DISTAL_2_CONTACT_1 = tfm.translation_matrix([0, DISTAL_LENGTH/2, 0])
DISTAL_2_CONTACT_2 = tfm.translation_matrix([0.0, DISTAL_LENGTH/2, 0])

CONTACT_PLANE_ROT = -np.pi/4
BASE_2_CONTACT_PARENT = tfm.translation_matrix([0.1, 0, 1.4])
plane_origin = tfm.translation_matrix([0, 0, 0])

base = vis["base"]
base.set_object(geom.Sphere(radius=0.08))

proximal_pivot = base["proximal_pivot"]
proximal_pivot["axis"].set_object(geom.triad())

proximal_body = proximal_pivot["proximal_body"]

proximal_body.set_transform(PROXIMAL_2_DISTAL_1)
proximal_body.set_object(geom.Cylinder(radius=FINGER_RADIUS, height=PROXIMAL_LENGTH))
proximal_body["axis"].set_object(geom.triad())

distal_init = proximal_body['distal_init']
distal_init.set_transform(PROXIMAL_2_DISTAL_2)

distal_pivot = distal_init['distal_pivot']
distal_pivot["axis"].set_object(geom.triad())
distal_pivot.set_object(geom.Sphere(radius=FINGER_RADIUS))

distal_body = distal_pivot["distal_body"]
distal_body.set_transform(DISTAL_2_CONTACT_1)
distal_body.set_object(geom.Cylinder(radius=FINGER_RADIUS, height=DISTAL_LENGTH))
distal_body["axis"].set_object(geom.triad())

contact_sphere = distal_body["contact_sphere"]
contact_sphere.set_transform(DISTAL_2_CONTACT_2)
contact_sphere.set_object(geom.Sphere(radius=0.08))

plane_parent = vis["plane_parent"]
plane_parent.set_transform(BASE_2_CONTACT_PARENT)
plane_moving = plane_parent["plane_parent"]
plane = plane_moving['plane']
plane.set_object(geom.Box(lengths=[2, 2, -.001]))
plane.set_transform(tfm.euler_matrix(0, CONTACT_PLANE_ROT, 0))
plane["axis"].set_object(geom.triad())




In [6]:
def rk4_step(f, x, dt):
    """
        Input:
            xdot = f(x) - function to be integrated, passed as f
            x - initial condition to the function
            dt - time step
        Output:
            x[t+dt]
    """
    # one step of runge-kutta integration
    k1 = f(x)
    k2 = f(x + dt*k1/2)
    k3 = f(x + dt*k2/2)
    k4 = f(x + dt*k3)
    xdot = x + 1/6*(k1+2*k2+2*k3+k4)*dt
    return xdot

def euler_origin_to_plane(roll, pitch, yaw, origin):
    pitch = -(np.pi/2 + pitch)
    x0, y0, z0 = origin

    A = math.cos(pitch) * math.cos(yaw)
    B = math.cos(pitch) * math.sin(yaw)
    C = math.sin(pitch)
    D = -(A * x0 + B * y0 + C * z0)

    return A, B, C, D

def distance_to_plane(point, plane):
    x, y, z = point
    A, B, C, D = plane

    numerator = A * x + B * y + C * z + D
    denominator = math.sqrt(A**2 + B**2 + C**2)

    distance = numerator / denominator
    return distance - FINGER_RADIUS

def phi_contact(q):
    _proximal_rot = SE3.from_rotation_and_translation(SO3.from_y_radians(q[0]), np.array([0, 0, 0])).as_matrix()
    _distal_rot = SE3.from_rotation_and_translation(SO3.from_z_radians(q[1]), np.array([0, 0, 0])).as_matrix()

    contact_point = BASE_2_PROXIMAL @ _proximal_rot @ PROXIMAL_2_DISTAL_1 @ PROXIMAL_2_DISTAL_2 @ _distal_rot @ DISTAL_2_CONTACT_1 @ DISTAL_2_CONTACT_2
    contact_plane = BASE_2_CONTACT_PARENT @ plane_origin
    contact_point_trans = SE3.from_matrix(contact_point)
    contact_plane_trans = SE3.from_matrix(contact_plane)

    plane = euler_origin_to_plane(0, CONTACT_PLANE_ROT, 0, contact_plane_trans.translation())
    return distance_to_plane(contact_point_trans.translation(), plane)

def phi_past(q):
    _proximal_rot = SE3.from_rotation_and_translation(SO3.from_y_radians(q[0]), np.array([0, 0, 0])).as_matrix()
    _distal_rot = SE3.from_rotation_and_translation(SO3.from_z_radians(q[1]), np.array([0, 0, 0])).as_matrix()

    contact_point = BASE_2_PROXIMAL @ _proximal_rot @ PROXIMAL_2_DISTAL_1 @ PROXIMAL_2_DISTAL_2 @ _distal_rot @ DISTAL_2_CONTACT_1 @ DISTAL_2_CONTACT_2
    contact_plane = BASE_2_CONTACT_PARENT @ plane_origin
    contact_point_trans = SE3.from_matrix(contact_point)
    contact_plane_trans = SE3.from_matrix(contact_plane)

    plane_x = contact_plane_trans.translation()[0] - 1 * np.cos(CONTACT_PLANE_ROT)
    contact_x = contact_point_trans.translation()[0] + FINGER_RADIUS

    plane_y = contact_plane_trans.translation()[2] + 1 * np.sin(CONTACT_PLANE_ROT)
    contact_y = contact_point_trans.translation()[2] + FINGER_RADIUS

    if (plane_x > contact_x or plane_y > contact_y):
        return 0
    return 1


def phi(q):
    return phi_contact(q) * phi_past(q)

contact_jac = jacfwd(phi)

In [7]:
def position_proximal(q):
    x1 = PROXIMAL_LENGTH * np.sin(q[0])
    y1 = PROXIMAL_LENGTH * np.cos(q[0])
    return np.asarray([x1, y1])

def position_distal(q):
    x1 = PROXIMAL_LENGTH * np.sin(q[0])
    y1 = PROXIMAL_LENGTH * np.cos(q[0])
    x2 =  x1 + DISTAL_LENGTH * -np.sin(q[0] + q[1])
    y2 =  y1 + DISTAL_LENGTH * -np.cos(q[0] + q[1])
    return np.asarray([x2,y2])

jac_position_proximal = jacfwd(position_proximal)
jac_position_distal = jacfwd(position_distal)

def KE_derived(q, qdot):
    v1 = jac_position_proximal(q)@qdot
    v2 = jac_position_distal(q)@qdot
    m1 = np.array([[MASS_kg_PROXIMAL, 0], [0, MASS_kg_PROXIMAL]])
    m2 = np.array([[MASS_kg_DISTAL, 0], [0, MASS_kg_DISTAL]])

    return 1/2 * v1.T @ m1 @ v1 + 1/2 * v2.T @ m1 @ v2

def PE_derived(q):
    return (PROXIMAL_SPRING_STIFFNESS * (q[0]**2) + DISTAL_SPRING_STIFFNESS * (q[1]**2))/2

def L_derived(q, qdot):
    return KE_derived(q, qdot) - PE_derived(q)

M_derived = jacfwd(jacfwd(L_derived, argnums=1), argnums=1)
C_derived = jacfwd(jacfwd(L_derived, argnums=1), argnums=0)
G_derived = jacfwd(L_derived)

k_contact = 10000

def f_finger(x):
    q,qdot = np.split(x, 2)

    qddot = -np.linalg.inv(M_derived(q, qdot)) @ (C_derived(q, qdot)@ qdot - G_derived(q, qdot)) + contact_jac(q).T * contact_force(q)
    xdot = np.array([qdot,qddot]).reshape((4,))
    return xdot

def contact_force(q):
    """
        Input: q, qdot (minus)
        Output: qdot (plus post impact)
    """
    ### FILL IN EQUATIONS HERE
    contact_pen = phi(q)
    contact_x = np.sin(CONTACT_PLANE_ROT) * contact_pen
    contact_y = np.cos(CONTACT_PLANE_ROT) * contact_pen
    lambda_val = np.array([np.maximum(0, k_contact * contact_x), np.maximum(0, -k_contact * contact_y)])

    return lambda_val

In [8]:
q = [0, 0.]
qdot = [0., 0.]
x0 = np.array(q+qdot)
# dt = 0.01
dt = 0.01

f_jit = jax.jit(f_finger)

x0s = []
for t in np.arange(0, 10, step=dt):
    plane_origin = tfm.translation_matrix([t/5, 0, 0])
    plane_moving.set_transform(plane_origin)

    x0 = rk4_step(f_finger, x0, dt)

    proximal_rot = tfm.euler_matrix(0, x0[0], 0)
    proximal_pivot.set_transform(proximal_rot)
    distal_rot = tfm.euler_matrix(0, 0, x0[1])
    distal_pivot.set_transform(distal_rot)
    x0s.append(x0)

    time.sleep(dt)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [13]:
for t in np.arange(0, 10, step=dt):
    plane_moving.set_transform(tfm.translation_matrix([t/5, 0, 0]))

    x0 = x_saved[int(t/dt)]
    proximal_pivot.set_transform(tfm.euler_matrix(0, x0[0], 0))
    distal_pivot.set_transform(tfm.euler_matrix(0, 0, x0[1]))

    time.sleep(dt)

In [9]:
# # Finger geometry
# vis.delete()


# proximal_pivot = vis["proximal_pivot"]
# proximal_pivot["axis"].set_object(geom.triad())

# proximal_body = proximal_pivot["proximal_body"]

# proximal_body.set_transform(tf.scale_matrix(0.1) @ tf.translation_matrix([0.2, -8, 75.65]) @ tf.euler_matrix(np.pi/2, 0, 0)  @ tf.euler_matrix(0, np.pi, 2.4))

# proximal_body["geom"].set_object(
#     geom.StlMeshGeometry.from_file("/home/josh/Documents/git/ENAS-773-code/midterm/proximal.stl")
# )

# distal_pivot_parent = proximal_body['distal_pivot_parent']
# distal_pivot_parent.set_transform(tf.euler_matrix(0, 0, np.pi/4) @ tf.translation_matrix(np.array([13,0,8])))
# distal_pivot_parent["axis"].set_object(geom.triad())

# distal_pivot = distal_pivot_parent['distal_pivot']

# distal_body = distal_pivot["distal_body"]
# distal_body["geom"].set_object(
#     geom.StlMeshGeometry.from_file("/home/josh/Documents/git/ENAS-773-code/midterm/finger_pp_t_B_round.stl")
# )
# distal_body.set_transform(tf.euler_matrix(0, np.pi/2, 0))

# contact_sphere = distal_body["contact_sphere"]
# contact_sphere.set_object(geom.Sphere(radius=7.2))
# contact_sphere.set_transform(tf.translation_matrix([0, 3.5, -42]))

# plane_parent = vis["plane_parent"]
# plane_parent.set_transform(tf.translation_matrix([-4, 0, 11]))
# plane_moving = plane_parent["plane_parent"]
# plane = plane_moving['plane']
# plane.set_object(geom.Box(lengths=[10, 10, .25]))
# plane.set_transform(tf.euler_matrix(0, -np.pi/4, 0))

# for t in np.arange(0, 10, step=0.01):
#     plane_moving.set_transform(tf.translation_matrix([t, 0, 0]))
#     time.sleep(0.01)